In [ ]:
#| default_exp peak_operations

In [ ]:
#| export
import pandas as pd
from typing import Literal
from pybedtools import BedTool
import pybedtools
from fastcore.meta import delegates
import numpy as np
#from lab_central_webapp.peak_to_gene import hg38_HiC_catalog

In [ ]:
#| export

def get_peak_info_df(peak_df):
    valid_bool = "peak" in peak_df or len(peak_df.columns.intersection(["chrom", "start", "stop"])) == 3 or len(peak_df.columns.intersection(["chrom", "start", "end"])) == 3 or len(peak_df.columns.intersection(["chr", "start", "end"])) == 3 or len(peak_df.columns.intersection(["chr", "start", "stop"])) == 3
    if not valid_bool:
        raise KeyError("Input peak dataframe need to have either 'peak' col or ['chrom', 'start', 'stop'] cols or ['chrom', 'start' 'end']")
    peak_df = peak_df.rename(columns={"stop": "end", "chr": "chrom"})
    if "peak" in peak_df:
        peak_df[["chrom", "start", "end"]] = peak_df["peak"].str.split("_", expand = True)
    else:
        peak_df["peak"] =  peak_df[["chrom", "start", "end"]].astype(str).apply("_".join, axis =1)
    
    peak_df = peak_df.set_index(["chrom", "start", "end"]).reset_index()
    peak_df[["start", "end"]] = peak_df[["start", "end"]].astype(int)
    peak_df = peak_df.sort_values(["chrom", "start", "end"])
    return peak_df

def intersect_bed_dfs(df_A, df_B, mode: Literal["intersect", "closest"] = "intersect", include_no_intersection_A=True, include_no_intersection_B=False, suffixes = ["peak_list", "HiC"],
                      #genome_build = "hg38",
                      ensembl_gene_TSS = None,
                      add_gene_info = True):
    columns_list = df_A.rename(columns = {"chrom": f"chrom_{suffixes[0]}", "start": f"start_{suffixes[0]}", "end": f"end_{suffixes[0]}", "peak": f"peak_{suffixes[0]}"}).columns.tolist() + df_B.rename(columns = {"chrom": f"chrom_{suffixes[1]}", "start": f"start_{suffixes[1]}", "end": f"end_{suffixes[1]}", "peak": f"peak_{suffixes[1]}"}).columns.tolist()
    df_A_bed = BedTool.from_dataframe(df_A.sort_values([ f"chrom",
                                                      f"start",
                                                      f"end"]))
    
    df_B_bed = BedTool.from_dataframe(df_B.sort_values([ f"chrom",
                                                      f"start",
                                                      f"end"]))
    
    if mode == "intersect":
        df = df_A_bed.intersect(df_B_bed, wa= True, wb = True, loj = include_no_intersection_A).to_dataframe(disable_auto_names= True).T.reset_index().T.reset_index(drop = True)
    elif mode == "closest":
        df = BedTool.from_dataframe(df_A)\
.closest(BedTool.from_dataframe(df_B)).to_dataframe()
       
    df.columns = columns_list
    
    if add_gene_info and "gene" in df:
        if ensembl_gene_TSS is None:
            print("No ensembl data provided, will not integrate gene info data")
        # if genome_build == "hg38":
        #     #ensembl_all_genes_TSS_hg38_df = hg38_HiC_catalog.catalog["ensembl_all_genes_TSS_hg38"].read_and_process()
        #     #ensembl_gene_TSS = ensembl_all_genes_TSS_hg38_df.set_index("gene").add_suffix("_TSS").reset_index()
        # if genome_build == "hg19":
        #     raise NotImplementedError
        else:
            df = df.merge(ensembl_gene_TSS, on = "gene", how ="left")
            df[["start_TSS", f"end_{suffixes[0]}"]] = df[["start_TSS", f"end_{suffixes[0]}"]].astype(float)
            df["distance_from_peak_to_TSS_in_kb"] = ((df["start_TSS"] - df[f"end_{suffixes[0]}"]).abs()/1000)
            df["gene_name_found_in_ensembl"] = df["distance_from_peak_to_TSS_in_kb"].notna()
    
       
    # if "distance" in df and maximum_bp_distance_filter:
    #     df = df.query("distance < @maximum_bp_distance_filter")
    pybedtools.cleanup(remove_all = True)
    return df



In [ ]:
#| export

class IntersectResult():
    def __init__(self, result_df, suffixes = ["peak_list", "HiC"]):
        self.raw_result_df = result_df
        chrom_A_col, start_A_col, end_A_col, peak_A_col, chrom_B_col, start_B_col, end_B_col, peak_B_col = [col + f"_{suffixes[0]}" for col in ["chrom", "start", "end", "peak"]]\
        + [col + f"_{suffixes[1]}" for col in ["chrom", "start", "end", "peak"]]
        
        if suffixes[1] == 'HiC':
            B_col = "gene"
        else:
            B_col = peak_B_col
            
        self.unmapped_peaks_df = self.raw_result_df.loc[self.raw_result_df[start_B_col].astype(int) < 0, :]
        self.mapped_peaks_df = self.raw_result_df.loc[self.raw_result_df[start_B_col].astype(int) > 0, :]
        self.mapped_peaks_df = self.mapped_peaks_df.drop_duplicates([peak_A_col, B_col])
        self.mapped_peaks_df.loc[:, [start_A_col, end_A_col,start_B_col, end_B_col]] = self.mapped_peaks_df.loc[:, [start_A_col, end_A_col,start_B_col, end_B_col]].astype(int)
        
        #harmonize column list for intersecting with lists in database that are not HiC
        for col in ["distance_from_peak_to_TSS_in_kb", "gene_name_found_in_ensembl", "file_name"]:
            if col not in self.mapped_peaks_df:
                self.mapped_peaks_df[col] = np.nan
            if col not in self.unmapped_peaks_df:
                self.unmapped_peaks_df[col] = np.nan
            
        self.unmapped_peaks_and_genes_df = self.unmapped_peaks_df.loc[:, [chrom_A_col, start_A_col, end_A_col, peak_A_col, B_col,  "file_name"]]
        self.unmapped_peaks_and_genes_df.columns = ["chrom", "start", "end", "peak",  B_col, "file_name"]
        
        


        self.mapped_peaks_and_genes_df = self.mapped_peaks_df.loc[:, [chrom_A_col, start_A_col, end_A_col, peak_A_col, B_col,  "file_name", "distance_from_peak_to_TSS_in_kb", "gene_name_found_in_ensembl"]]
        self.mapped_peaks_and_genes_df.columns = ["chrom", "start", "end", "peak",  B_col,  "file_name", "distance_from_peak_to_TSS_in_kb", "gene_name_found_in_ensembl"]

        
    
    

@pd.api.extensions.register_dataframe_accessor("genomic_range")
class GenomicRangeDataWrapper():
    def __init__(self, pandas_object):
        #self._obj = pandas_object
        #self.df = get_peak_info_df(pandas_object)
        self.df = pandas_object.sort_values(["chrom", "start", "end"])
        #self.region_df = BedTool.from_dataframe(self.df).merge().to_dataframe()
        pybedtools.cleanup(remove_all=True)
        #self.region_df["merged_region"] = self.region_df[["chrom", "start", "end"]].astype(str).apply("_".join, axis = 1)
    
    @delegates(intersect_bed_dfs, but = ["suffixes"])
    def intersect_peaks(self, df_A, suffixes = ["peak_list", "HiC"], **kwargs):
        peak_info_df = get_peak_info_df(df_A)
        #bed_file = pybedtools.BedTool.from_dataframe(peak_info_df)
        # intersected = (
        #     bed_file.intersect(pybedtools.BedTool.from_dataframe(self.df), wao=True)
        #     .to_dataframe()
        #     #.query("score != -1")
        # )
        intersected = intersect_bed_dfs(peak_info_df, self.df, suffixes = suffixes,
                                        **kwargs)
        
        return IntersectResult(intersected, suffixes = suffixes)
    
    #def closest(self, other)
    
    def combine_files_and_rename(self, new_name = None):
        if new_name is None:
            new_name = f"combined_{self.df.file_name.nunique()}_files={';'.join(self.df.file_name.unique())}"
        return self.df.drop_duplicates(["chrom", "start", "end", "gene"]).assign(file_name = new_name)
    
    @property
    def num_peaks(self):
        return self.df.peak.nunique()
    
    @property
    def num_peaks_per_chrom(self):
        return self.df.groupby("chrom").peak.nunique().sort_values()
    
    # @property
    # def num_regions(self):
    #     return self.region_df.merged_region.nunique()
    
    @property
    def num_regions_per_chrom(self):
        return self.region_df.groupby("chrom").merged_region.nunique().sort_values()
    
    @property
    def num_genes(self):
        return self.df.gene.nunique()
    
    @property
    def num_peaks_per_gene(self):
        return self.df.groupby("gene")["peak"].nunique().sort_values()
    
    @property
    def num_genes_per_peak(self):
        return self.df.groupby("peak")["gene"].nunique().sort_values()
    
    @property
    def peak_length_df(self):
        return self.df.assign(peak_length = self.df["end"] - self.df["start"])[["peak", "gene", "peak_length", "file_name"]].drop_duplicates()
    

/var/folders/t7/j0xwrw4n0hjdzyg_zwf_c_400000gp/T/ipykernel_6148/3283836915.py:40: UserWarning: registration of accessor <class '__main__.GenomicRangeDataWrapper'> under name 'genomic_range' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class GenomicRangeDataWrapper():
